In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf


Pre Process

In [19]:
data_dir = "archive\Garbage classification\Garbage classification"
img_height = 180
img_width = 180   

<>:1: SyntaxWarning: invalid escape sequence '\G'
<>:1: SyntaxWarning: invalid escape sequence '\G'
C:\Users\jiray\AppData\Local\Temp\ipykernel_20128\2288252158.py:1: SyntaxWarning: invalid escape sequence '\G'
  data_dir = "archive\Garbage classification\Garbage classification"


In [33]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,                  
  subset="training",                      
  seed=2222,                              
  image_size=(img_height, img_width),   
  batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",                
  seed=123,
  image_size=(img_height, img_width), 
  batch_size=32
)

Found 2527 files belonging to 6 classes.
Using 2022 files for training.
Found 2527 files belonging to 6 classes.
Using 505 files for validation.


In [36]:
class_names = train_ds.class_names
print(class_names)
print(f"class numbers {len(class_names)}")

['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
class numbers 6
